In [1]:
from IPython.core.magic import Magics, cell_magic, line_magic, magics_class
from pexpect import spawn

TIMEOUT = 300
PROG = "hive"
PROMPT = ["\r\n    > ", "\r\nhive> "]
QUIT = "quit;"


@magics_class
class Magic(Magics):
    def __init__(self, shell):
        super().__init__(shell)
        self.app = spawn(PROG, timeout=60)
        self.app.expect(PROMPT)

    @cell_magic
    def hive(self, line, cell):
        cell_lines = [cell_line.strip() for cell_line in cell.split("\n")]
        cell_lines = [cell_line for cell_line in cell_lines if cell_line != ""]
        for cell_line in cell_lines:
            self.app.sendline(cell_line)
            self.app.expect(PROMPT, timeout=TIMEOUT)
            output = self.app.before.decode()
            output = output.replace("\r\n", "\n")
            output = output.split("\n")
            output = [output_line.strip() for output_line in output]
            for output_line in output:
                if output_line not in cell_lines:
                    print(output_line)
        return None

    @line_magic
    def quit(self, line):
        self.app.sendline(QUIT)


def load_ipython_extension(ip):
    ip.register_magics(Magic(ip))


load_ipython_extension(ip=get_ipython())

In [ ]:
!hdfs dfs -copyFromLocal pregunta_02/data.tsv /tmp

In [ ]:
!hdfs dfs -ls /tmp

In [ ]:
!hdfs dfs -rm /tmp/data.tsv

In [ ]:
%%hive
DROP TABLE IF EXISTS data;

In [ ]:
%%hive
CREATE TABLE data (
letter STRING,
date_event STRING,
value INT
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY  '\t';

In [ ]:
%%hive
LOAD DATA INPATH '/tmp/data.tsv' OVERWRITE INTO TABLE data;
SELECT * FROM data;

In [ ]:
%%hive
SELECT DISTINCT(value)
FROM data
ORDER BY value LIMIT 5;

In [ ]:
%%hive
SELECT DISTINCT(letras) as let
FROM(
SELECT
explode(c5) as letras
FROM
tbl0
) w;
ORDER BY let

In [ ]:
!hdfs dfs -copyFromLocal pregunta_04/data0.csv /tmp

In [ ]:
!hdfs dfs -copyFromLocal pregunta_04/data1.csv /tmp

In [ ]:
!hdfs dfs -ls /tmp

In [ ]:
%%hive
DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA INPATH '/tmp/data0.csv' INTO TABLE tbl0;

In [ ]:
%%hive
SHOW TABLES;

In [ ]:
%%hive
SELECT DISTINCT(SUBSTRING(c4,0,4)) as c4 FROM tbl0;

In [ ]:
%%hive
SELECT explode(split(c5, ',')) AS word FROM tbl0 LIMIT 5;

In [ ]:
%%hive
SELECT otro, letras, count(*) as total FROM(
SELECT SUBSTRING(c4,0,4) as otro, letras FROM tbl0
LATERAL VIEW
    explode(c5) tbl0 as letras
ORDER BY otro, letras
) w
GROUP BY otro, letras;

In [ ]:
%%hive
SELECT COLLECT_SET(UPPER(letras)) FROM tbl0
LATERAL VIEW
    explode(c5) tbl0 as letras;

In [ ]:
%%hive
SELECT UPPER(CONCAT_WS(':',c5)) FROM tbl0;

In [ ]:
%%hive
SELECT c2, COLLECT_SET(c1) FROM tbl0
GROUP BY c2;

In [ ]:
%%hive
SELECT c2, COLLECT_SET(c1) as numeros FROM tbl0
GROUP BY c2
ORDER BY c2, numeros;

In [ ]:
%%hive
select c2 letra, sum(unal.value)
from tbl0
lateral view explode(c6) unal AS key, value
group by c2;


In [6]:
!hdfs dfs -copyFromLocal pregunta_09/data0.csv /tmp

In [7]:
%%hive
DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA INPATH '/tmp/data0.csv' INTO TABLE tbl0;

OK
Time taken: 0.028 seconds
OK
Time taken: 0.079 seconds
Loading data to table default.tbl0
OK
Time taken: 0.438 seconds


In [8]:
%%hive
select * from tbl0;

OK
1	D	4	2016-06-25	["a","e","c","d"]	{"bb":10,"dd":20,"cc":40}
2	C	4	2015-05-14	["a","c"]	{"dd":40,"bb":20,"cc":10}
3	D	6	2014-12-26	["b","d"]	{"aa":10,"bb":40}
4	D	5	2016-06-25	["a","c","e","b","d"]	{"bb":40,"dd":20,"aa":10,"cc":30}
5	C	7	2016-05-23	["d","e","c"]	{"cc":10,"aa":20}
6	A	2	2018-06-14	["a","d"]	{"aa":20}
7	B	3	2014-12-22	["a","e","d"]	{"cc":20}
8	C	6	2015-08-20	["d","a","c","e"]	{"cc":10,"aa":20}
9	B	3	2017-12-08	["b","a","c","e"]	{"cc":40,"dd":10,"aa":30,"bb":20}
10	B	7	2015-07-28	["c","d","e","a","b"]	{"aa":10,"dd":40,"cc":30}
Time taken: 0.155 seconds, Fetched: 10 row(s)


In [ ]:
!hdfs dfs -rm /tmp/data1.csv

In [2]:
!hdfs dfs -copyFromLocal pregunta_09/data1.csv /tmp

In [3]:
!hdfs dfs -ls /tmp

Found 3 items
-rw-r--r--   1 root supergroup        282 2023-06-03 19:22 /tmp/data1.csv
drwxrwx---   - root supergroup          0 2023-06-03 19:18 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2023-06-03 19:21 /tmp/hive


In [4]:
%%hive
DROP TABLE IF EXISTS tbl1;
CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA INPATH '/tmp/data1.csv' INTO TABLE tbl1;

OK
Time taken: 6.471 seconds
OK
Time taken: 0.631 seconds
Loading data to table default.tbl1
OK
Time taken: 1.036 seconds


In [5]:
%%hive
select c1 from tbl1;

OK
1
2
3
4
5
6
7
8
9
10
Time taken: 1.125 seconds, Fetched: 10 row(s)


In [9]:
!hdfs dfs -copyFromLocal pregunta_10/data.tsv /tmp

In [ ]:
!hdfs dfs -ls /tmp

In [10]:
%%hive
DROP TABLE IF EXISTS t0;
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';
LOAD DATA INPATH '/tmp/data.tsv' INTO TABLE t0;

OK
Time taken: 0.018 seconds
OK
Time taken: 0.098 seconds
Loading data to table default.t0
OK
Time taken: 0.287 seconds


In [14]:
%%hive
select unal.key, count(1)
from t0
lateral view explode(c3) unal AS key, value
group by unal.key;

Query ID = root_20230603195136_e82f3d37-0b35-4882-90c7-10b89564197d
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
set mapreduce.job.reduces=<number>
Starting Job = job_1685819908924_0003, Tracking URL = http://e29ee468bf5f:8088/proxy/application_1685819908924_0003/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1685819908924_0003
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2023-06-03 19:51:41,568 Stage-1 map = 0%,  reduce = 0%
2023-06-03 19:51:46,729 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 2.26 sec
2023-06-03 19:51:53,046 Stage-1 map = 100%,  reduce = 100%, Cumulative CPU 4.64 sec
MapReduce Total cumulative CPU time: 4 seconds 640

In [15]:
%%hive
select c1, size(c2) size_c2, size(c3) size_c3 from t0;

OK
E	3	5
A	3	4
B	4	4
A	2	4
C	4	4
A	2	5
A	3	6
B	2	3
E	4	6
B	4	6
C	4	5
C	4	3
D	4	5
E	2	3
B	2	5
D	2	4
E	3	6
D	2	3
E	4	3
E	2	3
E	2	3
E	3	3
D	3	3
A	3	5
E	2	6
E	3	6
A	3	3
E	3	5
A	2	5
C	4	6
A	2	5
D	2	6
E	2	4
B	3	6
B	3	5
D	2	3
B	2	5
C	4	3
E	2	3
E	3	3
Time taken: 0.134 seconds, Fetched: 40 row(s)


In [35]:
%%hive
select 
    explode(c2) as c2, unal.key 
    from(
        select 
        unal.key
    from t0
    lateral view explode(c3) unal AS key, value
    order by unal.key
) w
group by c2, unal.key;

FAILED: SemanticException [Error 10004]: Line 10:9 Invalid table alias or column reference 'c2': (possible column names are: key)


In [ ]:
%%hive
select c2, unal.key
from t0
lateral view explode(c3) unal AS key, value
order by unal.key;

In [ ]:
%%hive
select unal
from t0
lateral view explode(c2) t0 as unal
order by unal;


In [67]:
%%hive
select letra, unal.key, count(1) 
from(
select letra, c3
from t0
lateral view explode(c2) t0 as letra
order by letra) w
lateral view explode(c3) unal AS key, value
group by letra, unal.key;

Query ID = root_20230603205155_8301b923-4e9f-48dc-ac7c-ae84c025e444
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks determined at compile time: 1
In order to change the average load for a reducer (in bytes):
set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
set mapreduce.job.reduces=<number>
Starting Job = job_1685819908924_0013, Tracking URL = http://e29ee468bf5f:8088/proxy/application_1685819908924_0013/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1685819908924_0013
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2023-06-03 20:52:02,438 Stage-1 map = 0%,  reduce = 0%
2023-06-03 20:52:06,568 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 1.86 sec
2023-06-03 20:52:11,768 Stage-1 map = 100%,  reduce = 100%, Cumulative CPU 4.32 sec
MapReduce Total cumulative CPU time: 4 seconds 320 msec
Ended Job = j

In [81]:
%%hive
select t0.c1, t0.c2, t1.c4 from tbl0 t0
join (
select c1, c4 from tbl1
) t1
on (t0.c1 = t1.c1);

Query ID = root_20230603210955_bcf472cd-bea9-43dc-b5b8-8465b1c47f0c
Total jobs = 1
2023-06-03 21:10:03	Starting to launch local task to process map join;	maximum memory = 477626368
2023-06-03 21:10:05	Dump the side-table for tag: 1 with group count: 10 into file: file:/tmp/root/091c5dca-dfbb-4e37-a694-7faa28857a27/hive_2023-06-03_21-09-55_755_721422163089084542-1/-local-10004/HashTable-Stage-3/MapJoin-mapfile11--.hashtable
2023-06-03 21:10:05	Uploaded 1 File to: file:/tmp/root/091c5dca-dfbb-4e37-a694-7faa28857a27/hive_2023-06-03_21-09-55_755_721422163089084542-1/-local-10004/HashTable-Stage-3/MapJoin-mapfile11--.hashtable (673 bytes)
2023-06-03 21:10:05	End of local task; Time Taken: 1.79 sec.
Execution completed successfully
MapredLocal task succeeded
Launching Job 1 out of 1
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1685819908924_0016, Tracking URL = http://e29ee468bf5f:8088/proxy/application_1685819908924_0016/
Kill Command = /opt/hadoop/

In [92]:
%%hive
select t0.c1, t0.c2, t1.value from tbl0 t0
join (
select c1, unal.key, unal.value from tbl1
lateral view explode(c4) unal AS key, value
order by unal.key
) t1
on (t0.c1 = t1.c1 and t0.c2 = t1.key)
order by t0.c1;

Query ID = root_20230603212256_c832473f-29c0-4ae4-9fb6-d5f838c421f2
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks determined at compile time: 1
In order to change the average load for a reducer (in bytes):
set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
set mapreduce.job.reduces=<number>
Starting Job = job_1685819908924_0025, Tracking URL = http://e29ee468bf5f:8088/proxy/application_1685819908924_0025/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1685819908924_0025
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2023-06-03 21:23:05,155 Stage-1 map = 0%,  reduce = 0%
2023-06-03 21:23:11,503 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 4.51 sec
2023-06-03 21:23:17,709 Stage-1 map = 100%,  reduce = 100%, Cumulative CPU 6.66 sec
MapReduce Total cumulative CPU time: 6 seconds 660 msec
Ended Job = j

In [97]:
%%hive
select letra, unal.key, count(1) 
from(
select letra, c3
from t0
lateral view explode(c2) t0 as letra
order by letra) w
lateral view explode(c3) unal AS key, value
group by letra, unal.key;


Query ID = root_20230603214245_031c2a27-e448-4a25-9321-b4ef6294e6e4
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks determined at compile time: 1
In order to change the average load for a reducer (in bytes):
set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
set mapreduce.job.reduces=<number>
Starting Job = job_1685819908924_0029, Tracking URL = http://e29ee468bf5f:8088/proxy/application_1685819908924_0029/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1685819908924_0029
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2023-06-03 21:42:52,498 Stage-1 map = 0%,  reduce = 0%
2023-06-03 21:42:57,670 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 2.04 sec
2023-06-03 21:43:03,828 Stage-1 map = 100%,  reduce = 100%, Cumulative CPU 4.45 sec
MapReduce Total cumulative CPU time: 4 seconds 450 msec
Ended Job = j

In [96]:
%%hive
select letra, unal.key, count(1) 
from(
select letra, c3
from t0
lateral view explode(c2) t0 as letra
order by letra) w
lateral view explode(c3) unal AS key, value
group by letra, unal.key;

Query ID = root_20230603214148_ce45f7a5-e8ea-4c15-94a2-06de3818ef47
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks determined at compile time: 1
In order to change the average load for a reducer (in bytes):
set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
set mapreduce.job.reduces=<number>
Starting Job = job_1685819908924_0027, Tracking URL = http://e29ee468bf5f:8088/proxy/application_1685819908924_0027/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1685819908924_0027
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2023-06-03 21:41:57,084 Stage-1 map = 0%,  reduce = 0%
2023-06-03 21:42:02,279 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 2.28 sec
2023-06-03 21:42:08,471 Stage-1 map = 100%,  reduce = 100%, Cumulative CPU 4.95 sec
MapReduce Total cumulative CPU time: 4 seconds 950 msec
Ended Job = j